In [1]:
import boto3
import os
import pandas as pd

In [2]:
os.environ["AWS_PROFILE"] = "men"   
os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'

In [35]:
bucket_name = "men-prod-clean-data"

# ASIGNACION 1

In [36]:
prefixes = [
    "contenidos_formativos/APRENDER_DIGITAL_LIGERA_MOVIL/documents/",
    "contenidos_formativos/APRENDER_DIGITAL_LIGERA_MOVIL/pages/"
]

In [37]:
# Lista para almacenar archivos JSON encontrados
json_files = []

# Buscar archivos JSON en ambas carpetas
for prefix in prefixes:
    response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
    
    if 'Contents' in response:
        for obj in response['Contents']:
            if obj['Key'].endswith('.json'):  # Filtrar solo JSON
                json_files.append(obj['Key'])



In [38]:
# Contador de coincidencias
count = 0
count_preescolar = 0
count_basica_primaria = 0
count_basica_secundaria = 0
count_media = 0


# Buscar archivos JSON en ambas carpetas
for prefix in prefixes:
    response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
    
    if 'Contents' in response:
        for obj in response['Contents']:
            if obj['Key'].endswith('.json'):  # Filtrar solo JSON
                try:
                    # Leer JSON desde S3
                    json_obj = s3_client.get_object(Bucket=bucket_name, Key=obj['Key'])
                    json_content = json.load(json_obj['Body'])  # Convertir JSON a diccionario

                    # Extraer "page_url" o "url"
                    page_url = json_content.get("metadataAttributes", {}).get("page_url", "")
                    url = json_content.get("metadataAttributes", {}).get("url", "")

                    count += 1
                    # Verificar si "basica-secundaria" está presente en alguno de los valores
                    if "preescolar-educacion-inicial" in page_url or "preescolar-educacion-inicial" in url:
                        count_preescolar += 1
                    elif "basica-primaria" in page_url or "basica-primaria" in url:
                        count_basica_primaria += 1
                    elif "basica-secundaria" in page_url or "basica-secundaria" in url:
                        count_basica_secundaria += 1
                    elif "/media" in page_url or "/media" in url:
                        count_media += 1


                except Exception as e:
                    print(f"Error procesando {obj['Key']}: {e}")

# Imprimir el resultado final
print(f"\nTotal JSON: {count}")
print(f"\n'preescolar-educacion-inicial': {count_preescolar}")
print(f"\n'basica-primaria': {count_basica_primaria}")
print(f"\n'basica-secundaria': {count_basica_secundaria}")
print(f"\n'media': {count_media}")

Error procesando contenidos_formativos/APRENDER_DIGITAL_LIGERA_MOVIL/documents/2.%20Anexo%202%20-%20Orientaciones%20para%20Las%20Familias%20-%20Directiva%205.pdf.metadata.json: name 'json' is not defined
Error procesando contenidos_formativos/APRENDER_DIGITAL_LIGERA_MOVIL/documents/Fichero_DUA_1.pdf.metadata.json: name 'json' is not defined
Error procesando contenidos_formativos/APRENDER_DIGITAL_LIGERA_MOVIL/documents/Manual_instalacion_PWA.pdf.metadata.json: name 'json' is not defined
Error procesando contenidos_formativos/APRENDER_DIGITAL_LIGERA_MOVIL/documents/Retos%20para%20gigantes%204%C2%B0%20y%205.pdf.metadata.json: name 'json' is not defined
Error procesando contenidos_formativos/APRENDER_DIGITAL_LIGERA_MOVIL/documents/Retos%20para%20gigantes_1%C2%B0a_3%C2%B0_IIPARTE.pdf.metadata.json: name 'json' is not defined
Error procesando contenidos_formativos/APRENDER_DIGITAL_LIGERA_MOVIL/documents/Retos%20para%20gigantes_1%C2%B0a_3%C2%B0_IPARTE.pdf.metadata.json: name 'json' is not def

In [39]:
# Lista para guardar resultados
results = []

# Buscar archivos JSON en ambas carpetas
for prefix in prefixes:
    response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
    
    if 'Contents' in response:
        for obj in response['Contents']:
            if obj['Key'].endswith('.json'):  # Filtrar solo JSON
                try:
                    # Leer JSON desde S3
                    json_obj = s3_client.get_object(Bucket=bucket_name, Key=obj['Key'])
                    json_content = json.load(json_obj['Body'])  # Convertir JSON a diccionario

                    # Extraer "page_url" o "url"
                    page_url = json_content.get("metadataAttributes", {}).get("page_url", "")
                    url = json_content.get("metadataAttributes", {}).get("url", "")

                    # Aplicar la lógica para asignar age_from y age_to
                    if "preescolar-educacion-inicial" in page_url or "preescolar-educacion-inicial" in url:
                        age_from, age_to = 3, 6
                    elif "basica-primaria" in page_url or "basica-primaria" in url:
                        age_from, age_to = 6, 11
                    elif "basica-secundaria" in page_url or "basica-secundaria" in url:
                        age_from, age_to = 11, 15
                    elif "/media" in page_url or "/media" in url:
                        age_from, age_to = 15, 17
                    else:
                        age_from, age_to = 0, 99  # Valor por defecto

                    # Actualizar los valores en el JSON
                    json_content["metadataAttributes"]["age_from"] = age_from
                    json_content["metadataAttributes"]["age_to"] = age_to

                    # Guardar los resultados
                    results.append({
                        "Archivo JSON": obj['Key'],
                        "page_url": page_url,
                        "age_from": age_from,
                        "age_to": age_to
                    })

                    # Subir el JSON actualizado a S3
                    s3_client.put_object(
                        Bucket=bucket_name,
                        Key=obj['Key'],
                        Body=json.dumps(json_content, indent=4)
                    )
                    print(f"Actualizado: {obj['Key']}")

                except Exception as e:
                    print(f"Error procesando {obj['Key']}: {e}")

# Imprimir resumen
print("\nResumen de asignaciones:")
for result in results:
    print(f"{result['Archivo JSON']} → page_url: {result['page_url']}, age_from: {result['age_from']}, age_to: {result['age_to']}")

Error procesando contenidos_formativos/APRENDER_DIGITAL_LIGERA_MOVIL/documents/2.%20Anexo%202%20-%20Orientaciones%20para%20Las%20Familias%20-%20Directiva%205.pdf.metadata.json: name 'json' is not defined
Error procesando contenidos_formativos/APRENDER_DIGITAL_LIGERA_MOVIL/documents/Fichero_DUA_1.pdf.metadata.json: name 'json' is not defined
Error procesando contenidos_formativos/APRENDER_DIGITAL_LIGERA_MOVIL/documents/Manual_instalacion_PWA.pdf.metadata.json: name 'json' is not defined
Error procesando contenidos_formativos/APRENDER_DIGITAL_LIGERA_MOVIL/documents/Retos%20para%20gigantes%204%C2%B0%20y%205.pdf.metadata.json: name 'json' is not defined
Error procesando contenidos_formativos/APRENDER_DIGITAL_LIGERA_MOVIL/documents/Retos%20para%20gigantes_1%C2%B0a_3%C2%B0_IIPARTE.pdf.metadata.json: name 'json' is not defined
Error procesando contenidos_formativos/APRENDER_DIGITAL_LIGERA_MOVIL/documents/Retos%20para%20gigantes_1%C2%B0a_3%C2%B0_IPARTE.pdf.metadata.json: name 'json' is not def

# ASIGNACION 2

In [40]:
prefixes2 = [
    "contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/",
    "contenidos_formativos/CONTENIDOS_PARA_APRENDER/"
]

In [42]:
def list_s3_objects(bucket_name, prefixes, max_keys=20):
    s3 = boto3.client('s3')
    
    for prefix in prefixes:
        print(f"Objects in {prefix}:")
        response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix, MaxKeys=max_keys)
        
        if 'Contents' in response:
            for obj in response['Contents']:
                print(obj['Key'])
        else:
            print("No objects found.")
        
        print("-" * 40)

bucket_name = "men-prod-clean-data"
prefixes2 = [
    "contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/",
    "contenidos_formativos/CONTENIDOS_PARA_APRENDER/"
]

list_s3_objects(bucket_name, prefixes2)

Objects in contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/:
contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G01_U02_L06/MG_L_G01_U02_L06.txt
contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G01_U02_L06/MG_L_G01_U02_L06.txt.metadata.json
contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G01_U03_L05/MG_L_G01_U03_L05.txt
contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G01_U03_L05/MG_L_G01_U03_L05.txt.metadata.json
contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G01_U05_L01/MG_L_G01_U05_L01.txt
contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G01_U05_L01/MG_L_G01_U05_L01.txt.metadata.json
contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G01_U05_L02/MG_L_G01_U05_L02.txt
contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G01_U05_L02/MG_L_G01_U05_L02.txt.metadata.json
contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G01_U05_L03/MG_L_G01_U05_L03.txt
contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G01_U05_L03/MG

In [43]:
print(len("contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G01"))
print(len("contenidos_formativos/CONTENIDOS_PARA_APRENDER/MenuMedia/g10"))

60
60


In [44]:
def list_unique_prefixes(bucket_name, prefixes):
    s3 = boto3.client('s3')
    unique_prefixes = set()
    
    for prefix in prefixes:
        print(f"Checking objects in {prefix}...")
        response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
        
        if 'Contents' in response:
            for obj in response['Contents']:
                truncated_key = obj['Key'][:62]
                unique_prefixes.add(truncated_key)
        else:
            print(f"No objects found in {prefix}.")
        
    print("Unique first 60 characters of object keys:")
    for unique in sorted(unique_prefixes):
        print(unique)
    
    return unique_prefixes

list_unique_prefixes(bucket_name, prefixes2)


Checking objects in contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/...
Checking objects in contenidos_formativos/CONTENIDOS_PARA_APRENDER/...
Unique first 60 characters of object keys:
contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G01_U
contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G02_U
contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G03_U
contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G04_U
contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G05_U
contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G06_U
contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G07_U
contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G08_U
contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G09_U
contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G10_U
contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G11_U
contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_M_G01_U
contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_M

{'contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G01_U',
 'contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G02_U',
 'contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G03_U',
 'contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G04_U',
 'contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G05_U',
 'contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G06_U',
 'contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G07_U',
 'contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G08_U',
 'contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G09_U',
 'contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G10_U',
 'contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G11_U',
 'contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_M_G01_U',
 'contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_M_G02_U',
 'contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_M_G03_U',
 'contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_M_G04

In [45]:
import boto3
import json
import re

# Definir los rangos de edad basados en los grados
AGE_MAPPING = {
    "G01": (6, 7), "G02": (7, 8), "G03": (8, 9), "G04": (9, 10), "G05": (10, 11),
    "G06": (11, 12), "G07": (12, 13), "G08": (13, 14), "G09": (14, 15),
    "G10": (15, 16), "G11": (16, 17)
}

# Inicializar cliente S3
s3_client = boto3.client('s3')

# Lista para guardar resultados
results = []

# Función para asignar age_from y age_to basado en el nombre del archivo
def assign_age_from_filename(filename):
    match = re.search(r"[Gg](\d{1,2})", filename)
    if match:
        grade = f"G{int(match.group(1)):02d}"  # Asegurar formato GXX
        return AGE_MAPPING.get(grade, (0, 99))  # Valor por defecto si no se encuentra
    return (0, 99)


for prefix in prefixes2:
    response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
    
    if 'Contents' in response:
        for obj in response['Contents']:
            if obj['Key'].endswith('.json'):  # Filtrar solo JSON
                try:
                    # Leer JSON desde S3
                    json_obj = s3_client.get_object(Bucket=bucket_name, Key=obj['Key'])
                    json_content = json.load(json_obj['Body'])  # Convertir JSON a diccionario

                    # Asignar edades según el nombre del archivo
                    age_from, age_to = assign_age_from_filename(obj['Key'])
                    
                    # Actualizar los valores en el JSON
                    json_content.setdefault("metadataAttributes", {})
                    json_content["metadataAttributes"]["age_from"] = age_from
                    json_content["metadataAttributes"]["age_to"] = age_to

                    # Guardar los resultados
                    results.append({
                        "Archivo JSON": obj['Key'],
                        "age_from": age_from,
                        "age_to": age_to
                    })

                    # Subir el JSON actualizado a S3
                    s3_client.put_object(
                        Bucket=bucket_name,
                        Key=obj['Key'],
                        Body=json.dumps(json_content, indent=4)
                    )
                    print(f"Actualizado: {obj['Key']}")

                except Exception as e:
                    print(f"Error procesando {obj['Key']}: {e}")

# Imprimir resumen
print("\nResumen de asignaciones:")
for result in results:
    print(f"{result['Archivo JSON']} → age_from: {result['age_from']}, age_to: {result['age_to']}")

Actualizado: contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G01_U02_L06/MG_L_G01_U02_L06.txt.metadata.json
Actualizado: contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G01_U03_L05/MG_L_G01_U03_L05.txt.metadata.json
Actualizado: contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G01_U05_L01/MG_L_G01_U05_L01.txt.metadata.json
Actualizado: contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G01_U05_L02/MG_L_G01_U05_L02.txt.metadata.json
Actualizado: contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G01_U05_L03/MG_L_G01_U05_L03.txt.metadata.json
Actualizado: contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G01_U05_L05/MG_L_G01_U05_L05.txt.metadata.json
Actualizado: contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G01_U06_L01/MG_L_G01_U06_L01.txt.metadata.json
Actualizado: contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G01_U06_L02/MG_L_G01_U06_L02.txt.metadata.json
Actualizado: contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/m

# ASIGNACION 3

In [8]:
import boto3
import pandas as pd

# Configuración de los buckets y sus respectivos prefijos
buckets_and_prefixes = {
    "men-prod-clean-data": "contenidos_formativos/ENGLISH_FOR_COLOMBIA_ECOWEB/",
    "men-prod-raw-data": "ENGLISH_FOR_COLOMBIA_ECOWEB/"
}

# Crear cliente de S3
s3_client = boto3.client('s3')

# Función para listar TODOS los objetos dentro de un prefijo
def list_all_objects(bucket_name, prefix):
    try:
        paginator = s3_client.get_paginator('list_objects_v2')
        page_iterator = paginator.paginate(Bucket=bucket_name, Prefix=prefix)

        all_objects = []
        for page in page_iterator:
            if 'Contents' in page:
                all_objects.extend([obj['Key'] for obj in page['Contents'] if obj['Key'].endswith('.metadata.json')])

        return all_objects
    except Exception as e:
        print(f"⚠️ Error al listar objetos en {bucket_name}/{prefix}: {e}")
        return []

# Función para contar objetos dentro de las carpetas de primer orden
def count_objects_by_top_folder(bucket_name, prefix):
    all_objects = list_all_objects(bucket_name, prefix)
    folder_data = {}

    for obj_key in all_objects:
        # Obtener solo la carpeta de primer orden
        relative_path = obj_key.replace(prefix, '').split('/')
        if len(relative_path) > 1:  # Verifica que no sea un archivo en la raíz
            top_folder = relative_path[0] + '/'  # Obtener la carpeta principal
        else:
            top_folder = "(raíz)/"  # En caso de archivos en la raíz (poco probable)

        # Contar archivos en esa carpeta de primer orden
        if top_folder not in folder_data:
            folder_data[top_folder] = 0
        folder_data[top_folder] += 1

    return [{"carpeta": folder, "metadata_objetos": count} for folder, count in folder_data.items()]

# Obtener datos de TODAS las carpetas de primer orden
data_clean = count_objects_by_top_folder("men-prod-clean-data", buckets_and_prefixes["men-prod-clean-data"])
data_raw = count_objects_by_top_folder("men-prod-raw-data", buckets_and_prefixes["men-prod-raw-data"])

# Crear DataFrames
carpetas_clean = pd.DataFrame(data_clean)
carpetas_raw = pd.DataFrame(data_raw)



ModuleNotFoundError: No module named 'ace_tools'

In [9]:
carpetas_clean

,carpeta,metadata_objetos
0,about-bilingualism/,1
1,aboutfranco/,1
2,betheone/,3
3,eco_radio_teach/,208
4,evaluation-tools/,68
5,home-learn/,1
6,masterclass/,63
7,repository/,550
8,suggested-english-language-curriculum/,6
9,textbooks/,88


In [10]:
carpetas_raw

,carpeta,metadata_objetos
0,about-bilingualism/,1
1,aboutfranco/,1
2,betheone/,3
3,eco-radio-learn-2-temporada/,1
4,eco-radio-learn-3-temporada/,1
5,eco-radio-teach-temporada-2/,1
6,eco_radio_teach/,67
7,evaluation-tools/,36
8,home-learn/,1
9,masterclass/,1


In [34]:
carpetas_clean["objetos"].sum()

1851

In [11]:
import boto3
import json

# Configuración del bucket y prefijo
bucket_name = "men-prod-raw-data"
prefix = "ENGLISH_FOR_COLOMBIA_ECOWEB/"

# Crear cliente de S3
s3_client = boto3.client('s3')

# Función mejorada para obtener TODOS los archivos .metadata.json, incluyendo los de subcarpetas
def list_metadata_files(bucket, prefix):
    try:
        paginator = s3_client.get_paginator('list_objects_v2')
        page_iterator = paginator.paginate(Bucket=bucket, Prefix=prefix)

        metadata_files = []
        for page in page_iterator:
            if 'Contents' in page:
                # Filtrar solo los archivos que terminan en .metadata.json
                metadata_files.extend([obj['Key'] for obj in page['Contents'] if obj['Key'].endswith('.metadata.json')])

        if not metadata_files:
            print(f"⚠️ No se encontraron archivos en {bucket}/{prefix}")

        return metadata_files

    except s3_client.exceptions.NoSuchBucket:
        print(f"❌ El bucket {bucket} no existe.")
        return []
    except s3_client.exceptions.ClientError as e:
        error_code = e.response['Error']['Code']
        if error_code == "AccessDenied":
            print(f"🚫 Acceso denegado a {bucket}. Verifica permisos.")
        else:
            print(f"⚠️ Error desconocido con {bucket}: {e}")
        return []

# Función para verificar si un JSON tiene el campo "nivel_educativo"
def check_nivel_educativo(bucket, file_key):
    try:
        response = s3_client.get_object(Bucket=bucket, Key=file_key)
        content = response['Body'].read().decode('utf-8')  # Leer y decodificar el contenido
        json_content = json.loads(content)  # Convertir el contenido en un diccionario JSON

        # Verificar si el campo "nivel_educativo" está en "metadataAttributes"
        return "metadataAttributes" in json_content and "nivel_educativo" in json_content["metadataAttributes"]

    except Exception as e:
        print(f"⚠️ Error al leer {file_key}: {e}")
        return False

# Obtener nombres de archivos .metadata.json en TODAS las subcarpetas
metadata_files = list_metadata_files(bucket_name, prefix)

# Verificar cuántos tienen "nivel_educativo"
if metadata_files:
    total_files = len(metadata_files)
    files_with_nivel_educativo = sum(1 for file in metadata_files if check_nivel_educativo(bucket_name, file))

    # Calcular porcentaje
    percentage = (files_with_nivel_educativo / total_files) * 100 if total_files > 0 else 0

    print(f"📊 Total de archivos .metadata.json en todas las subcarpetas: {total_files}")
    print(f"✅ Archivos con 'nivel_educativo': {files_with_nivel_educativo} ({percentage:.2f}%)")
else:
    print(f"⚠️ No se encontraron archivos .metadata.json en {bucket_name}/{prefix}.")


📊 Total de archivos .metadata.json en todas las subcarpetas: 1514
✅ Archivos con 'nivel_educativo': 1514 (100.00%)


In [12]:
import boto3
import json

# Configuración del bucket y prefijo
bucket_name = "men-prod-raw-data"
prefix = "ENGLISH_FOR_COLOMBIA_ECOWEB/"

# Crear cliente de S3
s3_client = boto3.client('s3')

# Función mejorada para obtener TODOS los archivos .metadata.json, incluyendo los de subcarpetas
def list_metadata_files(bucket, prefix):
    try:
        paginator = s3_client.get_paginator('list_objects_v2')
        page_iterator = paginator.paginate(Bucket=bucket, Prefix=prefix)

        metadata_files = []
        for page in page_iterator:
            if 'Contents' in page:
                # Filtrar solo los archivos que terminan en .metadata.json
                metadata_files.extend([obj['Key'] for obj in page['Contents'] if obj['Key'].endswith('.metadata.json')])

        if not metadata_files:
            print(f"⚠️ No se encontraron archivos en {bucket}/{prefix}")

        return metadata_files

    except s3_client.exceptions.NoSuchBucket:
        print(f"❌ El bucket {bucket} no existe.")
        return []
    except s3_client.exceptions.ClientError as e:
        error_code = e.response['Error']['Code']
        if error_code == "AccessDenied":
            print(f"🚫 Acceso denegado a {bucket}. Verifica permisos.")
        else:
            print(f"⚠️ Error desconocido con {bucket}: {e}")
        return []

# Función para extraer los valores únicos de "nivel_educativo"
def get_nivel_educativo_values(bucket, file_key):
    try:
        response = s3_client.get_object(Bucket=bucket, Key=file_key)
        content = response['Body'].read().decode('utf-8')  # Leer y decodificar el contenido
        json_content = json.loads(content)  # Convertir el contenido en un diccionario JSON

        # Extraer el valor de "nivel_educativo" si está presente
        if "metadataAttributes" in json_content and "nivel_educativo" in json_content["metadataAttributes"]:
            return json_content["metadataAttributes"]["nivel_educativo"]
    
    except Exception as e:
        print(f"⚠️ Error al leer {file_key}: {e}")
    
    return None  # Si hay algún error, devolver None

# Obtener nombres de archivos .metadata.json en TODAS las subcarpetas
metadata_files = list_metadata_files(bucket_name, prefix)

# Extraer los valores únicos de "nivel_educativo"
nivel_educativo_values = set()

if metadata_files:
    for file in metadata_files:
        value = get_nivel_educativo_values(bucket_name, file)
        if value:  # Si el valor no es None, añadirlo al set
            nivel_educativo_values.add(value)

    # Imprimir los valores únicos encontrados
    print("\n📊 Valores únicos en 'nivel_educativo':")
    for value in sorted(nivel_educativo_values):
        print(f"   - {value}")
else:
    print(f"⚠️ No se encontraron archivos .metadata.json en {bucket_name}/{prefix}.")



📊 Valores únicos en 'nivel_educativo':
   - Educación básica y secundaria


In [13]:
import boto3

# Configuración de los buckets y prefijos
buckets_and_prefixes = {
    "men-prod-raw-data": "ENGLISH_FOR_COLOMBIA_ECOWEB/",
    "men-prod-clean-data": "contenidos_formativos/ENGLISH_FOR_COLOMBIA_ECOWEB/"
}

# Crear cliente de S3
s3_client = boto3.client('s3')

# Función mejorada para obtener TODOS los archivos .metadata.json, incluyendo los de subcarpetas
def list_metadata_files(bucket, prefix):
    try:
        paginator = s3_client.get_paginator('list_objects_v2')
        page_iterator = paginator.paginate(Bucket=bucket, Prefix=prefix)

        metadata_files = set()
        for page in page_iterator:
            if 'Contents' in page:
                # Extraer nombres de archivos sin el prefijo
                files = {obj['Key'].replace(prefix, '') for obj in page['Contents'] if obj['Key'].endswith('.metadata.json')}
                metadata_files.update(files)

        if not metadata_files:
            print(f"⚠️ No se encontraron archivos en {bucket}/{prefix}")

        return metadata_files

    except s3_client.exceptions.NoSuchBucket:
        print(f"❌ El bucket {bucket} no existe.")
        return set()
    except s3_client.exceptions.ClientError as e:
        error_code = e.response['Error']['Code']
        if error_code == "AccessDenied":
            print(f"🚫 Acceso denegado a {bucket}. Verifica permisos.")
        else:
            print(f"⚠️ Error desconocido con {bucket}: {e}")
        return set()

# Obtener los nombres de archivos en ambos buckets (ahora incluyendo subcarpetas)
metadata_raw = list_metadata_files("men-prod-raw-data", buckets_and_prefixes["men-prod-raw-data"])
metadata_clean = list_metadata_files("men-prod-clean-data", buckets_and_prefixes["men-prod-clean-data"])

# Calcular intersección (archivos en ambos buckets)
common_files = metadata_raw.intersection(metadata_clean)

# Imprimir resultados
print(f"\n📊 Total de archivos .metadata.json en men-prod-raw-data (todas las subcarpetas): {len(metadata_raw)}")
print(f"📊 Total de archivos .metadata.json en men-prod-clean-data (todas las subcarpetas): {len(metadata_clean)}")
print(f"✅ Archivos presentes en ambos buckets: {len(common_files)}")



📊 Total de archivos .metadata.json en men-prod-raw-data (todas las subcarpetas): 1514
📊 Total de archivos .metadata.json en men-prod-clean-data (todas las subcarpetas): 989
✅ Archivos presentes en ambos buckets: 664


In [14]:
import boto3
import json

# Configuración del bucket y prefijo
bucket_name = "men-prod-clean-data"
prefix = "contenidos_formativos/ENGLISH_FOR_COLOMBIA_ECOWEB/"

# Crear cliente de S3
s3_client = boto3.client('s3')

# Función mejorada para obtener TODOS los archivos .metadata.json, incluyendo los de subcarpetas
def list_metadata_files(bucket, prefix):
    try:
        paginator = s3_client.get_paginator('list_objects_v2')
        page_iterator = paginator.paginate(Bucket=bucket, Prefix=prefix)

        metadata_files = []
        for page in page_iterator:
            if 'Contents' in page:
                # Filtrar solo los archivos que terminan en .metadata.json
                metadata_files.extend([obj['Key'] for obj in page['Contents'] if obj['Key'].endswith('.metadata.json')])

        if not metadata_files:
            print(f"⚠️ No se encontraron archivos en {bucket}/{prefix}")

        return metadata_files

    except s3_client.exceptions.NoSuchBucket:
        print(f"❌ El bucket {bucket} no existe.")
        return []
    except s3_client.exceptions.ClientError as e:
        error_code = e.response['Error']['Code']
        if error_code == "AccessDenied":
            print(f"🚫 Acceso denegado a {bucket}. Verifica permisos.")
        else:
            print(f"⚠️ Error desconocido con {bucket}: {e}")
        return []

# Función para verificar si un JSON tiene los campos "age_from" y "age_to"
def check_age_fields(bucket, file_key):
    try:
        response = s3_client.get_object(Bucket=bucket, Key=file_key)
        content = response['Body'].read().decode('utf-8')  # Leer y decodificar el contenido
        json_content = json.loads(content)  # Convertir el contenido en un diccionario JSON

        # Verificar si el campo "metadataAttributes" contiene "age_from" y "age_to"
        return "metadataAttributes" in json_content and "age_from" in json_content["metadataAttributes"] and "age_to" in json_content["metadataAttributes"]
    
    except Exception as e:
        print(f"⚠️ Error al leer {file_key}: {e}")
        return False

# Obtener nombres de archivos .metadata.json en TODAS las subcarpetas
metadata_files = list_metadata_files(bucket_name, prefix)

# Verificar cuántos tienen "age_from" y "age_to"
if metadata_files:
    total_files = len(metadata_files)
    files_with_age_fields = sum(1 for file in metadata_files if check_age_fields(bucket_name, file))

    # Calcular porcentaje
    percentage = (files_with_age_fields / total_files) * 100 if total_files > 0 else 0

    print(f"\n📊 Total de archivos .metadata.json en {bucket_name} (todas las subcarpetas): {total_files}")
    print(f"✅ Archivos con 'age_from' y 'age_to': {files_with_age_fields} ({percentage:.2f}%)")

    # Si hay archivos sin estos campos, muestra ejemplos
    if files_with_age_fields < total_files:
        missing_files = [file for file in metadata_files if not check_age_fields(bucket_name, file)]
        print("\n⚠️ Ejemplo de archivos que NO tienen 'age_from' o 'age_to':")
        for file in missing_files[:10]:  # Muestra hasta 10 ejemplos
            print(f"   - {file}")
else:
    print(f"⚠️ No se encontraron archivos .metadata.json en {bucket_name}/{prefix}.")



📊 Total de archivos .metadata.json en men-prod-clean-data (todas las subcarpetas): 989
✅ Archivos con 'age_from' y 'age_to': 989 (100.00%)


In [15]:
import boto3
import json

# Configuración de los buckets y prefijos
buckets_and_prefixes = {
    "men-prod-raw-data": "ENGLISH_FOR_COLOMBIA_ECOWEB/",
    "men-prod-clean-data": "contenidos_formativos/ENGLISH_FOR_COLOMBIA_ECOWEB/"
}

# Crear cliente de S3
s3_client = boto3.client('s3')

# Función mejorada para obtener TODOS los archivos, incluyendo los de subcarpetas
def list_metadata_files(bucket, prefix):
    try:
        paginator = s3_client.get_paginator('list_objects_v2')
        page_iterator = paginator.paginate(Bucket=bucket, Prefix=prefix)

        metadata_files = set()
        for page in page_iterator:
            if 'Contents' in page:
                # Extraer nombres de archivos sin el prefijo
                files = {obj['Key'].replace(prefix, '') for obj in page['Contents'] if obj['Key'].endswith('.metadata.json')}
                metadata_files.update(files)
        
        if not metadata_files:
            print(f"⚠️ No se encontraron archivos en {bucket}/{prefix}")

        return metadata_files
    
    except s3_client.exceptions.NoSuchBucket:
        print(f"❌ El bucket {bucket} no existe.")
        return set()
    except s3_client.exceptions.ClientError as e:
        error_code = e.response['Error']['Code']
        if error_code == "AccessDenied":
            print(f"🚫 Acceso denegado a {bucket}. Verifica permisos.")
        else:
            print(f"⚠️ Error desconocido con {bucket}: {e}")
        return set()

# Función para verificar si un JSON en men-prod-raw-data tiene "nivel_educativo" como "Educación básica y secundaria"
def check_nivel_educativo(bucket, file_key):
    try:
        response = s3_client.get_object(Bucket=bucket, Key=file_key)
        content = response['Body'].read().decode('utf-8')  # Leer y decodificar el contenido
        json_content = json.loads(content)  # Convertir el contenido en un diccionario JSON

        # Verificar si el campo "metadataAttributes" contiene "nivel_educativo" con el valor correcto
        return "metadataAttributes" in json_content and json_content["metadataAttributes"].get("nivel_educativo") == "Educación básica y secundaria"
    
    except Exception as e:
        print(f"⚠️ Error al leer {file_key}: {e}")
        return False

# Función para actualizar age_from y age_to en los archivos de men-prod-clean-data
def update_age_fields(bucket, file_key, age_from, age_to):
    try:
        # Descargar el archivo original
        response = s3_client.get_object(Bucket=bucket, Key=file_key)
        content = response['Body'].read().decode('utf-8')
        json_content = json.loads(content)

        # Asegurar que metadataAttributes existe
        if "metadataAttributes" not in json_content:
            json_content["metadataAttributes"] = {}

        # Actualizar los valores de age_from y age_to
        json_content["metadataAttributes"]["age_from"] = age_from
        json_content["metadataAttributes"]["age_to"] = age_to

        # Guardar el archivo actualizado en S3 (sobreescribiendo)
        updated_content = json.dumps(json_content, indent=4, ensure_ascii=False)
        s3_client.put_object(Bucket=bucket, Key=file_key, Body=updated_content, ContentType="application/json")

        print(f"✅ Archivo actualizado: {file_key} -> age_from: {age_from}, age_to: {age_to}")

    except Exception as e:
        print(f"⚠️ Error al actualizar {file_key}: {e}")

# Obtener la lista de archivos en ambos buckets (ahora incluyendo subcarpetas)
metadata_raw = list_metadata_files("men-prod-raw-data", buckets_and_prefixes["men-prod-raw-data"])
metadata_clean = list_metadata_files("men-prod-clean-data", buckets_and_prefixes["men-prod-clean-data"])

# Procesar los archivos en men-prod-clean-data
for file_name in metadata_clean:
    clean_file_key = buckets_and_prefixes["men-prod-clean-data"] + file_name
    raw_file_key = buckets_and_prefixes["men-prod-raw-data"] + file_name

    if file_name in metadata_raw and check_nivel_educativo("men-prod-raw-data", raw_file_key):
        # Si el archivo existe en ambos y tiene "nivel_educativo": "Educación básica y secundaria"
        update_age_fields("men-prod-clean-data", clean_file_key, age_from=6, age_to=15)
    else:
        # Si el archivo no está en ambos o no tiene el nivel educativo correcto
        update_age_fields("men-prod-clean-data", clean_file_key, age_from=0, age_to=99)


✅ Archivo actualizado: contenidos_formativos/ENGLISH_FOR_COLOMBIA_ECOWEB/textbooks/WTG8_M4_UnitTest.pdf.metadata.json -> age_from: 0, age_to: 99
✅ Archivo actualizado: contenidos_formativos/ENGLISH_FOR_COLOMBIA_ECOWEB/eco_radio_teach/ECO KIDS_CÁPSULA 13_STORYTELLING.txt.metadata.json -> age_from: 0, age_to: 99
✅ Archivo actualizado: contenidos_formativos/ENGLISH_FOR_COLOMBIA_ECOWEB/repository/twinkle-twinkle-little-star-song-with-audio-lyrics-and-poster/Twinkle-Twinkle-Little-Star-Lyrics.pdf.metadata.json -> age_from: 6, age_to: 15
✅ Archivo actualizado: contenidos_formativos/ENGLISH_FOR_COLOMBIA_ECOWEB/eco_radio_teach/ECO TEENS_C╡PSULA 3_STORYTELLING.txt.metadata.json -> age_from: 0, age_to: 99
✅ Archivo actualizado: contenidos_formativos/ENGLISH_FOR_COLOMBIA_ECOWEB/repository/teaching-guide-my-family/9_AR_PT_SC_2019.docx.pdf.metadata.json -> age_from: 6, age_to: 15
✅ Archivo actualizado: contenidos_formativos/ENGLISH_FOR_COLOMBIA_ECOWEB/repository/learning-guide-a-common-day-in-life/